In [2]:
#import scipy as sc
import pandas as pnd

from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer


In [3]:
#inits
tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()

In [38]:
people = pnd.read_csv('abc.csv',header=0,index_col=False)

In [45]:
# Prepare my_df, copy names from people to here
DataTaken=len(people)
DataTaken=20
my_df = pnd.DataFrame()
#my_df['name'] = people['name'][0:DataTaken]
#my_df['name'] = people['name']
#for 
#people.head()
for i in range(DataTaken):
    if (pnd.isnull(people.loc(i)['text'][i])):
        people['text'][i-1] =  "{}{}".format(people['text'][i-1],people['name'][i])
        people.drop(people.index[1], inplace=True)
#people.drop(people.index[1], inplace=True)
print DataTaken
print len(people)
people.head()


KeyError: 'the label [text] is not in the [index]'

In [5]:
# define function to remove stop_words, stemming, tokenizer, lower etc
def documentCleaner(text):
    #convert text to lower
    raw = text.lower()
    
    #create token from raw data
    tokens = tokenizer.tokenize(raw)
    
    #Stop word treatment
    en_stop = get_stop_words('en')
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    #Stemming
    #text = [p_stemmer.stem(i) for i in tokens]
    text = [p_stemmer.stem(i) for i in stopped_tokens]
    
    return text
    

In [6]:
# creating cleaner dataframe
#my_df['text'] = people['text'][0:500].map(lambda x : documentCleaner(x))
my_df['text'] = people['text'][0:DataTaken].map(lambda x : documentCleaner(x))


In [7]:
#import math
#from textblob import TextBlob as tb

#def tf(word, blob):
#    return blob.words.count(word) / len(blob.words)

#def n_containing(word, bloblist):
#    return sum(1 for blob in bloblist if word in blob)


#def idf(word, bloblist):
#    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

#def tfidf(word, blob, bloblist):
#    return tf(word, blob) * idf(word, bloblist)

In [8]:
#my_df['tf_idf'] = my_df[0:500]['text'].map(lambda x: (sorted(tfidf(word, x, my_df[0:500]['text']),key=lambda y:y[1], reverse=True) for word in x))
#for i, blob in enumerate(my_df[0:500]['text']):
#    print("Top words in document {}".format(i + 1))
#    scores = {word: tfidf(word, tb(blob), my_df['text'][0:500].map(lambda x : documentCleaner(x))) for word in tb(blob).words}
#    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
#    for word, score in sorted_words[:3]:
#        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

In [9]:
#print tb("sahil makkar the great")

In [10]:
#from textblob.tokenizers import WordTokenizer 
#wt = WordTokenizer()
#print (tb(people['text'][1]))

#for i,data in enumerate(people['text'][:10]):
#    my_df['tf_idf'][i] = tb(people['text'][i])

#print my_df.head(10)
#people['text'][0:10].map(lambda x: tb(x,tokenizer=wt.tokenize(wt, x.lower())))

In [11]:
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import bigrams, trigrams
import math
import numpy as np


stopwords = nltk.corpus.stopwords.words('portuguese')
tokenizer = RegexpTokenizer("[\w’]+", flags=re.UNICODE)


def freq(word, doc):
    return doc.count(word)


def word_count(doc):
    return len(doc)


def tf(word, doc):
    return (freq(word, doc) / float(word_count(doc)))


def num_docs_containing(word, list_of_docs):
    count = 0
    for document in list_of_docs:
        if freq(word, document) > 0:
            count += 1
    return 1 + count


def idf(word, list_of_docs):
    return math.log(len(list_of_docs) /
            float(num_docs_containing(word, list_of_docs)))


def tf_idf(word, doc, list_of_docs):
    return (tf(word, doc) * idf(word, list_of_docs))

#Compute the frequency for each term.
vocabulary = []
docs = {}
all_tips = []
#del my_df['tf_idf']
my_df['tf-idf'] = ""
for rowIdx, tip in enumerate(people['text'][0:10]):
    tokens = tokenizer.tokenize(tip)

    #bi_tokens = bigrams(tokens)
    #tri_tokens = trigrams(tokens)
    tokens = [token.lower() for token in tokens if len(token) > 2]
    tokens = [token for token in tokens if token not in get_stop_words('en')]

    #bi_tokens = [' '.join(token).lower() for token in bi_tokens]
    #bi_tokens = [token for token in bi_tokens if token not in stopwords]

    #tri_tokens = [' '.join(token).lower() for token in tri_tokens]
    #tri_tokens = [token for token in tri_tokens if token not in stopwords]

    final_tokens = []
    final_tokens.extend(tokens)
    #final_tokens.extend(bi_tokens)
    #final_tokens.extend(tri_tokens)
    docs[tip] = {'freq': {}, 'tf': {}, 'idf': {},
                        'tf-idf': {}, 'tokens': []}

    for token in final_tokens:
        #The frequency computed for each tip
        docs[tip]['freq'][token] = freq(token, final_tokens)
        #The term-frequency (Normalized Frequency)
        docs[tip]['tf'][token] = tf(token, final_tokens)
        docs[tip]['tokens'] = final_tokens

    vocabulary.append(final_tokens)

for rowIdx,doc in enumerate(docs):
    listTmp = []
    for token in docs[doc]['tf']:
        #The Inverse-Document-Frequency
        docs[doc]['idf'][token] = idf(token, vocabulary)
        #The tf-idf
        ##docs[doc]['tf-idf'][token] = tf_idf(token, docs[doc]['tokens'], vocabulary)
        listTmp.append(tf_idf(token, docs[doc]['tokens'], vocabulary))
        
    #print ("doc{}\n".format(doc))
    ##my_df.loc[rowIdx]['tf-idf'] = np.array(docs[doc]['tf-idf'])
    my_df.loc[rowIdx]['tf-idf'] = listTmp
    
    #print "========================================="
    ##print rowIdx,my_df.loc[rowIdx]['name'],docs[doc]['tf-idf']

#Now let's find out the most relevant words by tf-idf.
words = {}
for doc in docs:
    for token in docs[doc]['tf-idf']:
        if token not in words:
            words[token] = docs[doc]['tf-idf'][token]
        else:
            if docs[doc]['tf-idf'][token] > words[token]:
                words[token] = docs[doc]['tf-idf'][token]

    #print doc
    for token in docs[doc]['tf-idf']:
        print token, docs[doc]['tf-idf'][token]

for item in sorted(words.items(), key=lambda x: x[1], reverse=True):
    print "%f <= %s" % (item[1], item[0])

In [12]:
my_df['tf-idf'][0]

[0.009406037533796376,
 0.012573733690891408,
 0.012573733690891408,
 0.009406037533796376,
 0.010830424696249145,
 0.012573733690891408,
 0.012573733690891408,
 0.0008231290285767684,
 0.009406037533796376,
 0.050294934763565634,
 0.012573733690891408,
 0.012573733690891408,
 0.007158521342766837,
 0.025147467381782817,
 0.012573733690891408,
 0.025147467381782817,
 0.012573733690891408,
 0.04295112805660102,
 0.012573733690891408,
 0.012573733690891408,
 0.037721201072674224,
 0.012573733690891408,
 0.037721201072674224,
 0.012573733690891408,
 0.012573733690891408,
 0.012573733690891408,
 0.009406037533796376,
 0.012573733690891408,
 0.0016462580571535367,
 0.012573733690891408,
 0.037721201072674224,
 0.037721201072674224,
 0.012573733690891408,
 0.002786522999521347,
 0.012573733690891408,
 0.009406037533796376,
 0.050294934763565634,
 0.012573733690891408,
 0.012573733690891408,
 0.012573733690891408,
 0.012573733690891408,
 0.035792606713834184,
 0.009406037533796376,
 0.0125737

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfModel = TfidfVectorizer(min_df=1,tokenizer=documentCleaner,stop_words="english")

#print my_df['text'][0]
X = tfidfModel.fit_transform(people['text'][0:DataTaken])

In [14]:
print X.get_shape()
print tfidfModel.get_feature_names()

(500, 17119)
[u'0', u'000', u'005', u'020', u'04', u'05', u'06', u'09', u'0fer', u'1', u'10', u'100', u'1000', u'10000', u'100000', u'1000000', u'100001996', u'10000m', u'10000mgrage', u'1000m', u'1000plu', u'100200and', u'100db', u'100m', u'100th', u'101', u'1011', u'10119', u'10140', u'102', u'10200', u'1021', u'1023fm', u'1025', u'1028', u'102nd', u'103', u'10372', u'103rd', u'1044', u'1049fm', u'105', u'1059', u'106', u'107', u'109', u'109th', u'10in', u'10k', u'10m', u'10part', u'10th', u'10thbest', u'11', u'110', u'1107', u'1111', u'1115', u'112th', u'113', u'1135', u'115', u'1151', u'116', u'117', u'1178', u'1184', u'119th', u'11am', u'11th', u'12', u'120', u'120000', u'12021974', u'121', u'1211', u'1214153', u'122', u'12252', u'123', u'12307', u'1233', u'124', u'125', u'125th', u'129', u'129000', u'1298', u'12m', u'12month', u'12page', u'12th', u'13', u'130', u'131', u'1310', u'1312', u'1313', u'132274', u'133892', u'134', u'135', u'135000', u'136', u'138', u'1391', u'13px', u'

In [15]:
max(X.A[0])



0.46192938487176416

In [62]:
from sklearn import neighbors
knn=neighbors.KNeighborsClassifier(n_neighbors=10)

In [63]:
knn.fit(X.A,my_df['name'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [ ]:
pnd.set_option('display.max_colwidth', -1)
index=4
print people.loc[index]
a = knn.predict(X.A[index])
print people[people['name'] == a[0]]
print a[0]
print a.shape
#print ("{}".format(people[people['name'] == 'Ben Stratton']['text']))
#print knn.classes_

In [71]:
print knn.predict(X.A[3:6])
print people.loc[3:6]['name']
#print knn.classes_

print knn.get_params
#for i,value in enumerate(X.A[0]):
#    if (value != 0):
#        print "i{}:value{}".format(i,value)

['Alan Baxter (author)' 'Angus Clark' 'Bo Hampton']
3    Franz Rottensteiner
4    G-Enka             
5    Sam Henderson      
6    Aaron LaCrate      
Name: name, dtype: object
<bound method KNeighborsClassifier.get_params of KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')>


In [8]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(50, 0.4)
neigh.fit(X.A)
print neigh.kneighbors(X.A[0],return_distance=False)

for i in xrange(5):
    print "prediction neighbour of ::: {}".format(people.loc[i]['name'])
    print neigh.kneighbors(X.A[0])
    #print [people.loc[j]['text'] for j in neigh.kneighbors(X.A[i],return_distance=False)]

C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[[   0 1347  806  621 1180 1690   99 2601 1221 1361 1621 2038 2203  329
   314  635  118 2018 2199 1143 2350 2546  881 2849 2739 2590 2810 2025
  1189 2253  770 1233 2501 2969  152 1973 1754 1797 2300  914 1605  558
  1021 1056 2250 2967 1237 1927 1521 2940]]
prediction neighbour of ::: Digby Morrell
(array([[ 0.        ,  1.06111336,  1.10020379,  1.10933376,  1.13243909,
         1.14475385,  1.17065349,  1.17996441,  1.19375125,  1.19883384,
         1.20252906,  1.2054032 ,  1.21293875,  1.21629031,  1.21633648,
         1.21739669,  1.22702733,  1.22727042,  1.23655165,  1.24235746,
         1.2447501 ,  1.24546446,  1.24825216,  1.24844031,  1.25150328,
         1.25231037,  1.25934712,  1.25939953,  1.26077315,  1.2616155 ,
         1.26408751,  1.26577909,  1.2687818 ,  1.27138992,  1.27172603,
         1.27262309,  1.27397465,  1.2754769 ,  1.27716386,  1.27769893,
         1.27782203,  1.27828   ,  1.27870945,  1.28111574,  1.28143573,
         1.28266935,  1.28391521,  1.284

C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)



prediction neighbour of ::: Harpdog Brown
(array([[ 0.        ,  1.06111336,  1.10020379,  1.10933376,  1.13243909,
         1.14475385,  1.17065349,  1.17996441,  1.19375125,  1.19883384,
         1.20252906,  1.2054032 ,  1.21293875,  1.21629031,  1.21633648,
         1.21739669,  1.22702733,  1.22727042,  1.23655165,  1.24235746,
         1.2447501 ,  1.24546446,  1.24825216,  1.24844031,  1.25150328,
         1.25231037,  1.25934712,  1.25939953,  1.26077315,  1.2616155 ,
         1.26408751,  1.26577909,  1.2687818 ,  1.27138992,  1.27172603,
         1.27262309,  1.27397465,  1.2754769 ,  1.27716386,  1.27769893,
         1.27782203,  1.27828   ,  1.27870945,  1.28111574,  1.28143573,
         1.28266935,  1.28391521,  1.28459172,  1.28595145,  1.28677395]]), array([[   0, 1347,  806,  621, 1180, 1690,   99, 2601, 1221, 1361, 1621,
        2038, 2203,  329,  314,  635,  118, 2018, 2199, 1143, 2350, 2546,
         881, 2849, 2739, 2590, 2810, 2025, 1189, 2253,  770, 1233, 2501,
 

C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)



prediction neighbour of ::: G-Enka
(array([[ 0.        ,  1.06111336,  1.10020379,  1.10933376,  1.13243909,
         1.14475385,  1.17065349,  1.17996441,  1.19375125,  1.19883384,
         1.20252906,  1.2054032 ,  1.21293875,  1.21629031,  1.21633648,
         1.21739669,  1.22702733,  1.22727042,  1.23655165,  1.24235746,
         1.2447501 ,  1.24546446,  1.24825216,  1.24844031,  1.25150328,
         1.25231037,  1.25934712,  1.25939953,  1.26077315,  1.2616155 ,
         1.26408751,  1.26577909,  1.2687818 ,  1.27138992,  1.27172603,
         1.27262309,  1.27397465,  1.2754769 ,  1.27716386,  1.27769893,
         1.27782203,  1.27828   ,  1.27870945,  1.28111574,  1.28143573,
         1.28266935,  1.28391521,  1.28459172,  1.28595145,  1.28677395]]), array([[   0, 1347,  806,  621, 1180, 1690,   99, 2601, 1221, 1361, 1621,
        2038, 2203,  329,  314,  635,  118, 2018, 2199, 1143, 2350, 2546,
         881, 2849, 2739, 2590, 2810, 2025, 1189, 2253,  770, 1233, 2501,
        

In [17]:
import matplotlib.pyplot as plt

def plot_surface(est, x_1, x_2, ax=None, threshold=0.0, contourf=False):
    """Plots the decision surface of ``est`` on features ``x1`` and ``x2``. """
    xx1, xx2 = np.meshgrid(np.linspace(x_1.min(), x_1.max(), 100), 
                           np.linspace(x_2.min(), x_2.max(), 100))
    # plot the hyperplane by evaluating the parameters on the grid
    X_pred = np.c_[xx1.ravel(), xx2.ravel()]  # convert 2d grid into seq of points
    if hasattr(est, 'predict_proba'):  # check if ``est`` supports probabilities
        # take probability of positive class
        pred = est.predict_proba(X_pred)[:, 1]
    else:
        pred = est.predict(X_pred)
    Z = pred.reshape((100, 100))  # reshape seq to grid
    if ax is None:
        ax = plt.gca()
    # plot line via contour plot

    if contourf:
        ax.contourf(xx1, xx2, Z, levels=np.linspace(0, 1.0, 10), cmap=plt.cm.RdBu, alpha=0.6)
    ax.contour(xx1, xx2, Z, levels=[threshold], colors='black')
    ax.set_xlim((x_1.min(), x_1.max()))
    ax.set_ylim((x_2.min(), x_2.max()))
    
def plot_datasets(est=None):
    """Plotsthe decision surface of ``est`` on each of the three datasets. """
    fig, axes = plt.subplots(1, 3, figsize=(10, 4))
    for (name, ds), ax in zip(datasets.iteritems(), axes):
        X_train = ds['X_train']
        y_train = ds['y_train']
        X_test = ds['X_test']
        y_test = ds['y_test']

        # plot test lighter than training
        cm_bright = ListedColormap(['#FF0000', '#0000FF'])
        # Plot the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6)
        # plot limits
        ax.set_xlim(X_train[:, 0].min(), X_train[:, 0].max())
        ax.set_ylim(X_train[:, 1].min(), X_train[:, 1].max())
         # no ticks
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_ylabel('$x_1$')
        ax.set_xlabel('$x_0$')
        ax.set_title(name)
        if est is not None:
            est.fit(X_train, y_train)
            plot_surface(est, X_train[:, 0], X_train[:, 1], ax=ax, threshold=0.5, contourf=True)
            err = (y_test != est.predict(X_test)).mean()
            ax.text(0.88, 0.02, '%.2f' % err, transform=ax.transAxes)

    fig.subplots_adjust(left=.02, right=.98)

In [24]:
from sklearn.neighbors import KNeighborsClassifier

est = KNeighborsClassifier(n_neighbors=50)

estfit = est.fit(X.A,my_df['name'])


#plt.plot(X.A)
#plot_datasets(est)

In [39]:
estfit.score(X.A[10:50],people['name'][10:50])
#people[people['name'] == 'Barack Obama']

0.050000000000000003

In [33]:
import numpy as np
estfit.predict(X.A[0])
#plt.plot(est,X.A)

XA=X.A
y=my_df['name']
h=.02

x_min, x_max = XA[:, 0].min() - 1, XA[:, 0].max() + 1
y_min, y_max = XA[:, 1].min() - 1, XA[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
print xx,yy

print xx.ravel(), yy.ravel()
Z = estfit.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(XA[:, 0], XA[:, 1], c=y, cmap=cmap_bold)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
#plt.title("3-Class classification (k = %i, weights = '%s')"
#              % (n_neighbors, weights))

plt.show()

[[-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ]
 [-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ]
 [-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ]
 ..., 
 [-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ]
 [-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ]
 [-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ]] [[-1.   -1.   -1.   ..., -1.   -1.   -1.  ]
 [-0.98 -0.98 -0.98 ..., -0.98 -0.98 -0.98]
 [-0.96 -0.96 -0.96 ..., -0.96 -0.96 -0.96]
 ..., 
 [ 1.04  1.04  1.04 ...,  1.04  1.04  1.04]
 [ 1.06  1.06  1.06 ...,  1.06  1.06  1.06]
 [ 1.08  1.08  1.08 ...,  1.08  1.08  1.08]]
[-1.   -0.98 -0.96 ...,  1.06  1.08  1.1 ] [-1.   -1.   -1.   ...,  1.08  1.08  1.08]


C:\Users\Dikshit\Anaconda4\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: query data dimension must match training data dimension

In [118]:
# k-Nearest Neighbor
from sklearn import datasets
#from sklearn import metrics
#from sklearn.neighbors import KNeighborsClassifier
# load iris the datasets
dataset = datasets.load_iris()

In [120]:
print(dataset.data)
print dataset.target


[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]
 [ 5.4  3.9  1.7  0.4]
 [ 4.6  3.4  1.4  0.3]
 [ 5.   3.4  1.5  0.2]
 [ 4.4  2.9  1.4  0.2]
 [ 4.9  3.1  1.5  0.1]
 [ 5.4  3.7  1.5  0.2]
 [ 4.8  3.4  1.6  0.2]
 [ 4.8  3.   1.4  0.1]
 [ 4.3  3.   1.1  0.1]
 [ 5.8  4.   1.2  0.2]
 [ 5.7  4.4  1.5  0.4]
 [ 5.4  3.9  1.3  0.4]
 [ 5.1  3.5  1.4  0.3]
 [ 5.7  3.8  1.7  0.3]
 [ 5.1  3.8  1.5  0.3]
 [ 5.4  3.4  1.7  0.2]
 [ 5.1  3.7  1.5  0.4]
 [ 4.6  3.6  1.   0.2]
 [ 5.1  3.3  1.7  0.5]
 [ 4.8  3.4  1.9  0.2]
 [ 5.   3.   1.6  0.2]
 [ 5.   3.4  1.6  0.4]
 [ 5.2  3.5  1.5  0.2]
 [ 5.2  3.4  1.4  0.2]
 [ 4.7  3.2  1.6  0.2]
 [ 4.8  3.1  1.6  0.2]
 [ 5.4  3.4  1.5  0.4]
 [ 5.2  4.1  1.5  0.1]
 [ 5.5  4.2  1.4  0.2]
 [ 4.9  3.1  1.5  0.1]
 [ 5.   3.2  1.2  0.2]
 [ 5.5  3.5  1.3  0.2]
 [ 4.9  3.1  1.5  0.1]
 [ 4.4  3.   1.3  0.2]
 [ 5.1  3.4  1.5  0.2]
 [ 5.   3.5  1.3  0.3]
 [ 4.5  2.3  1.3  0.3]
 [ 4.4  3.2  1.3  0.2]
 [ 5.   3.5